In [ ]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import tqdm
import sln_letter_fit
from sln_letter_fit import FitParams
from sln_stroke_fit import OptimizationLoggingParams

%load_ext autoreload
%autoreload 1
%aimport sln_letter_fit

# Loading and Plotting Utilities

In [ ]:
DT = 1./120
def load_letter(letter='D'):
    with np.load('../all_letters.npz', allow_pickle=True) as data:
        return data[letter]
def split_curve(curve, debug=False):
    v = np.diff(curve[:, 1:], axis=0)
    speed2 = np.sum(np.square(v), axis=1)
    speed2_smooth = scipy.signal.savgol_filter(speed2, 5, 1)
    # find local minima in speeds
    splits, _ = scipy.signal.find_peaks(-speed2_smooth, width=0.02 * 120)

    if debug:
        plt.subplot(121)
        # plt.plot(speed2, 'k-')
        plt.plot(speed2_smooth, '-', linewidth=1)
        plt.plot(splits, speed2[splits], 'r*')
        plt.subplot(122)
        plt.plot(curve[:, 1], curve[:, 2])
        plt.plot(curve[splits, 1], curve[splits, 2], 'r*')

    splits = [0, *splits, curve.shape[0] - 1]
    return [curve[begin:end, :] for begin, end in zip(splits[:-1], splits[1:])]
def curve2strokes(curve, debug=False):
    t = np.arange(0, curve.shape[0] * DT, DT).reshape(-1, 1)
    return split_curve(np.hstack((t, curve)), debug=debug)
def load_strokes(letter='D', index=1, debug=False):
    letter_data = load_letter(letter)
    if index is None:  # return all strokes
        return [curve2strokes(letter_data[i], debug=debug) for i in range(1, len(letter_data), 2)]
    return curve2strokes(letter_data[index], debug=debug)
# Debug:
# load_strokes('A', 1, debug=True);
plt.figure(figsize=(10, 5))
load_strokes('A', index=None, debug=True);
plt.show()

In [ ]:
def plot_letter(ax, strokes, sol):
    txy_gt = np.vstack(strokes)
    txy_params = sol['txy_from_params']
    ax.plot(txy_gt[:, 1], txy_gt[:, 2], 'k.', label='Mocap data')
    ax.plot(txy_params[:, 1], txy_params[:, 2], 'r-', label='Optimized SLN curve', linewidth=1)
    ax.plot(sol['txy'][:, 1], sol['txy'][:, 2], 'r.')
    ax.axis('equal')
    ax.legend()

# Fit and Plot the letter 'D'

In [ ]:
strokes = load_strokes('D', 1)

In [ ]:
sol, history, fitter, _ = sln_letter_fit.fit_letter(
    strokes,
    fit_params=FitParams(max_iters=50),
    optimization_logging_params=OptimizationLoggingParams(log_optimization_values=False,
                                                          progress_bar_class=tqdm.tqdm_notebook))


In [ ]:
fig, ax = plt.subplots()
plot_letter(ax, strokes, sol)
plt.show()

In [ ]:
def fit_and_plot_letter(ax, letter, num_strokes=None, num_trajectories=1, max_iters=100):
    all_trajectories = load_strokes(letter, index=None)
    if num_trajectories is None:
        num_trajectories = len(all_trajectories)
    all_trajectories = all_trajectories[:min(num_trajectories, len(all_trajectories))]
    for traji, strokes in enumerate(all_trajectories):
        if num_strokes is None:
            num_strokes = len(strokes)
        sol, history, fitter, _ = sln_letter_fit.fit_letter(
            strokes[:num_strokes],
            fit_params=FitParams(max_iters=max_iters),
            optimization_logging_params=OptimizationLoggingParams(
                log_optimization_values=False,
                progress_bar_class=tqdm.tqdm_notebook,
                progress_bar_description='Fitting Letter {:}, stroke{:}'.format(letter, traji)))
        plot_letter(ax, strokes, sol)
    return sol
# test
fig, ax = plt.subplots()
fit_and_plot_letter(ax, 'D', num_strokes=None, num_trajectories=None, max_iters=100);

# Fit and plot lots of letters

In [ ]:
def fit_and_plot_letters(letters, num_strokes=None, max_iters=25):
    fig, axes = plt.subplots(1, len(letters), figsize=(20, 5))
    for ax, letter in zip(axes, letters):
        fit_and_plot_letter(ax, letter, num_strokes=num_strokes, max_iters=max_iters)
fit_and_plot_letters('ABCD', 2, max_iters=100)

In [ ]:
letter = 'A'
strokes = load_strokes(letter)
sol, history, fitter, _ = sln_letter_fit.fit_letter(
    strokes,
    fit_params=FitParams(max_iters=50),
    optimization_logging_params=OptimizationLoggingParams(
        log_optimization_values=False,
        progress_bar_class=tqdm.tqdm_notebook,
        progress_bar_description='Fitting Letter ' + letter))
plot_letter(ax, strokes, sol)
solutions[letter] = sol